In [384]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 100)

## Import Master Data Frames

In [385]:
sens = pd.read_csv('./data/senate_races_cleaned.csv')
govs = pd.read_csv('./data/governors_races_cleaned.csv')

sens.drop('Unnamed: 0', inplace=True, axis=1)
govs.drop('Unnamed: 0', inplace=True, axis=1)

In [386]:
govs.head()

,race_id,office,loc_date_id,state,abbrev,year,GOP_win,winner,rival,pred_GOP,pred_DEM,unopposed,inc_GOP_running,inc_DEM_running
0,AL_gov_1978,AL_gov,AL_1978,Alabama,AL,1978,0,Fob James,H Guy Hunt,0,1,0,0,0
1,AL_gov_1982,AL_gov,AL_1982,Alabama,AL,1982,0,George Wallace,Emory Folmar,0,1,0,0,0
2,AL_gov_1986,AL_gov,AL_1986,Alabama,AL,1986,1,H Guy Hunt,Bill Baxley,0,1,0,0,0
3,AL_gov_1990,AL_gov,AL_1990,Alabama,AL,1990,0,Paul Hubbert,Unopposed,1,0,1,0,0
4,AL_gov_1994,AL_gov,AL_1994,Alabama,AL,1994,1,Fob James,Unopposed,0,1,1,0,0


In [387]:
sens.head()

,race_id,office,loc_date_id,state,abbrev,year,GOP_win,winner,rival,pred_GOP,pred_DEM,unopposed,inc_GOP_running,inc_DEM_running
0,NY_sen_1976,NY_sen,NY_1976,New_York,NY,1976,0,Daniel P Moynihan,James L Buckley,0,0,0,0,1
1,MN_sen_1996,MN_sen,MN_1996,Minnesota,MN,1996,0,Paul Wellstone Democratic,Rudy Boschwitz,0,0,0,0,1
2,AL_sen_1978,AL_sen,AL_1978,Alabama,AL,1978,0,Howell Heflin,Jerome B Couch,0,1,0,0,0
3,AL_sen_1980,AL_sen,AL_1980,Alabama,AL,1980,1,Jeremiah Denton,Jim Folsom Jr,0,1,0,0,0
4,AL_sen_1984,AL_sen,AL_1984,Alabama,AL,1984,0,Howell Heflin,Albert L Smith Jr,0,1,0,0,1


In [388]:
sen_states = sens.abbrev.unique()
gov_states = govs.abbrev.unique()
print(len(sen_states))
print(len(gov_states))
print(sen_states)
print(gov_states)

50
50
['NY' 'MN' 'AL' 'AK' 'AZ' 'AR' 'CA' 'CO' 'CT' 'DE' 'FL' 'GA' 'HI' 'ID'
 'IL' 'IN' 'IA' 'KY' 'LA' 'ME' 'MD' 'MA' 'MI' 'MS' 'MO' 'MT' 'NE' 'NV'
 'NH' 'NJ' 'NM' 'NC' 'ND' 'OH' 'OK' 'OR' 'PA' 'RI' 'SC' 'SD' 'TN' 'TX'
 'UT' 'VT' 'VA' 'WA' 'WV' 'WI' 'WY' 'KS']
['AL' 'AK' 'AZ' 'AR' 'CA' 'CO' 'CT' 'DE' 'FL' 'GA' 'HI' 'ID' 'IL' 'IN'
 'IA' 'KS' 'KY' 'LA' 'ME' 'MD' 'MA' 'MI' 'MN' 'MS' 'MO' 'MT' 'NE' 'NV'
 'NY' 'NH' 'NJ' 'NM' 'NC' 'ND' 'OH' 'OK' 'OR' 'PA' 'RI' 'SC' 'SD' 'TN'
 'TX' 'UT' 'VT' 'VA' 'WA' 'WV' 'WI' 'WY']


## Join with Pres Approval Data

#### Import and Join

In [389]:
approval = pd.read_csv('./data/pres_approval.csv')
approval.head()

,president,pres_party,prez_GOP,month,year,month_year,approve,disapprove,unsure
0,Ford,R,1,10,1975,10_1975,44,43,12
1,Ford,R,1,10,1975,10_1975,46,40,13
2,Ford,R,1,10,1975,10_1975,47,36,16
3,Ford,R,1,10,1974,10_1974,53,28,17
4,Ford,R,1,10,1974,10_1974,52,28,19


In [390]:
approval_join = approval.groupby('month_year')[['prez_GOP','approve', 'disapprove', 'unsure', 'year']].mean()
approval_join.reset_index(drop=True, inplace=True)
approval_join.head(10)

,prez_GOP,approve,disapprove,unsure,year
0,1.0,52.500000,28.000000,18.000000,1974.0
1,1.0,45.666667,39.666667,13.666667,1975.0
2,0.0,53.000000,29.333333,16.333333,1977.0
3,0.0,49.000000,36.000000,14.000000,1978.0
4,0.0,30.000000,55.666667,12.333333,1979.0
5,1.0,41.000000,47.000000,10.000000,1982.0
6,1.0,45.000000,43.000000,10.000000,1983.0
7,1.0,55.500000,33.500000,9.000000,1984.0
8,1.0,62.000000,29.000000,8.000000,1985.0
9,1.0,64.000000,28.000000,7.000000,1986.0


In [391]:
govs_with_approval = pd.merge(govs, approval_join, on='year')
sens_with_approval = pd.merge(sens, approval_join, on='year')

#### Engineer Features

In [392]:
govs_with_approval.head()

,race_id,office,loc_date_id,state,abbrev,year,GOP_win,winner,rival,pred_GOP,pred_DEM,unopposed,inc_GOP_running,inc_DEM_running,prez_GOP,approve,disapprove,unsure
0,AL_gov_1978,AL_gov,AL_1978,Alabama,AL,1978,0,Fob James,H Guy Hunt,0,1,0,0,0,0.0,49.0,36.0,14.0
1,AK_gov_1978,AK_gov,AK_1978,Alaska,AK,1978,0,Walter Joseph Hickel,Chancy Croft,1,0,0,0,0,0.0,49.0,36.0,14.0
2,AZ_gov_1978,AZ_gov,AZ_1978,Arizona,AZ,1978,1,Evan Mecham,V Gene Lewter,0,1,0,0,0,0.0,49.0,36.0,14.0
3,AR_gov_1978,AR_gov,AR_1978,Arkansas,AR,1978,0,Bill Clinton,Lynn Lowe,0,1,0,0,0,0.0,49.0,36.0,14.0
4,CA_gov_1978,CA_gov,CA_1978,California,CA,1978,1,Evelle J Younger,Ed Clark,0,1,0,0,0,0.0,49.0,36.0,14.0


In [393]:
sens_with_approval.head()

,race_id,office,loc_date_id,state,abbrev,year,GOP_win,winner,rival,pred_GOP,pred_DEM,unopposed,inc_GOP_running,inc_DEM_running,prez_GOP,approve,disapprove,unsure
0,MN_sen_1996,MN_sen,MN_1996,Minnesota,MN,1996,0,Paul Wellstone Democratic,Rudy Boschwitz,0,0,0,0,1,0.0,56.0,35.0,8.5
1,AL_sen_1996,AL_sen,AL_1996,Alabama,AL,1996,1,Jeff Sessions,Roger Bedford,0,1,0,0,0,0.0,56.0,35.0,8.5
2,AR_sen_1996,AR_sen,AR_1996,Arkansas,AR,1996,1,Tim Hutchinson,Winston Bryant,0,1,0,0,0,0.0,56.0,35.0,8.5
3,DE_sen_1996,DE_sen,DE_1996,Delaware,DE,1996,0,Joe Biden,Raymond J Clatworthy,0,1,0,0,1,0.0,56.0,35.0,8.5
4,GA_sen_1996,GA_sen,GA_1996,Georgia,GA,1996,0,Max Cleland,Guy Millner,0,1,0,0,0,0.0,56.0,35.0,8.5


In [394]:
def approval_engineer(df):
    n_rows = df.shape[0]
    
    df['approval_effects_GOP'] = 0
    df['approval_effects_DEM'] = 0
    df['approval_gap'] = df['approve'] - df['disapprove']
    
    for i in range(n_rows):
        if df.loc[i, 'prez_GOP'] == 1:
            df.loc[i, 'approval_effects_GOP'] = df.loc[i, 'approval_gap']
        else:
            df.loc[i, 'approval_effects_DEM'] = df.loc[i, 'approval_gap']
        
    df.drop(['approval_gap', 'approve', 'disapprove', 'unsure'], inplace=True, axis=1)
    return df

In [395]:
govs_with_approval = approval_engineer(govs_with_approval)
sens_with_approval = approval_engineer(sens_with_approval)

In [396]:
govs_with_approval.head()

,race_id,office,loc_date_id,state,abbrev,year,GOP_win,winner,rival,pred_GOP,pred_DEM,unopposed,inc_GOP_running,inc_DEM_running,prez_GOP,approval_effects_GOP,approval_effects_DEM
0,AL_gov_1978,AL_gov,AL_1978,Alabama,AL,1978,0,Fob James,H Guy Hunt,0,1,0,0,0,0.0,0.0,13.0
1,AK_gov_1978,AK_gov,AK_1978,Alaska,AK,1978,0,Walter Joseph Hickel,Chancy Croft,1,0,0,0,0,0.0,0.0,13.0
2,AZ_gov_1978,AZ_gov,AZ_1978,Arizona,AZ,1978,1,Evan Mecham,V Gene Lewter,0,1,0,0,0,0.0,0.0,13.0
3,AR_gov_1978,AR_gov,AR_1978,Arkansas,AR,1978,0,Bill Clinton,Lynn Lowe,0,1,0,0,0,0.0,0.0,13.0
4,CA_gov_1978,CA_gov,CA_1978,California,CA,1978,1,Evelle J Younger,Ed Clark,0,1,0,0,0,0.0,0.0,13.0


In [397]:
sens_with_approval.head()

,race_id,office,loc_date_id,state,abbrev,year,GOP_win,winner,rival,pred_GOP,pred_DEM,unopposed,inc_GOP_running,inc_DEM_running,prez_GOP,approval_effects_GOP,approval_effects_DEM
0,MN_sen_1996,MN_sen,MN_1996,Minnesota,MN,1996,0,Paul Wellstone Democratic,Rudy Boschwitz,0,0,0,0,1,0.0,0.0,21.0
1,AL_sen_1996,AL_sen,AL_1996,Alabama,AL,1996,1,Jeff Sessions,Roger Bedford,0,1,0,0,0,0.0,0.0,21.0
2,AR_sen_1996,AR_sen,AR_1996,Arkansas,AR,1996,1,Tim Hutchinson,Winston Bryant,0,1,0,0,0,0.0,0.0,21.0
3,DE_sen_1996,DE_sen,DE_1996,Delaware,DE,1996,0,Joe Biden,Raymond J Clatworthy,0,1,0,0,1,0.0,0.0,21.0
4,GA_sen_1996,GA_sen,GA_1996,Georgia,GA,1996,0,Max Cleland,Guy Millner,0,1,0,0,0,0.0,0.0,21.0


In [398]:
# reset to default names
sens = sens_with_approval
govs = govs_with_approval

In [399]:
sens.head()

,race_id,office,loc_date_id,state,abbrev,year,GOP_win,winner,rival,pred_GOP,pred_DEM,unopposed,inc_GOP_running,inc_DEM_running,prez_GOP,approval_effects_GOP,approval_effects_DEM
0,MN_sen_1996,MN_sen,MN_1996,Minnesota,MN,1996,0,Paul Wellstone Democratic,Rudy Boschwitz,0,0,0,0,1,0.0,0.0,21.0
1,AL_sen_1996,AL_sen,AL_1996,Alabama,AL,1996,1,Jeff Sessions,Roger Bedford,0,1,0,0,0,0.0,0.0,21.0
2,AR_sen_1996,AR_sen,AR_1996,Arkansas,AR,1996,1,Tim Hutchinson,Winston Bryant,0,1,0,0,0,0.0,0.0,21.0
3,DE_sen_1996,DE_sen,DE_1996,Delaware,DE,1996,0,Joe Biden,Raymond J Clatworthy,0,1,0,0,1,0.0,0.0,21.0
4,GA_sen_1996,GA_sen,GA_1996,Georgia,GA,1996,0,Max Cleland,Guy Millner,0,1,0,0,0,0.0,0.0,21.0


In [400]:
sen_states = sens.abbrev.unique()
gov_states = govs.abbrev.unique()
print(len(sen_states))
print(len(gov_states))
print(sen_states)
print(gov_states)

50
50
['MN' 'AL' 'AR' 'DE' 'GA' 'IL' 'IA' 'LA' 'MA' 'MI' 'MT' 'NE' 'NJ' 'RI'
 'WV' 'AK' 'CO' 'ID' 'KS' 'KY' 'ME' 'MS' 'NH' 'NM' 'NC' 'OK' 'OR' 'SC'
 'SD' 'TN' 'TX' 'VA' 'WY' 'HI' 'IN' 'CA' 'CT' 'FL' 'MD' 'NV' 'ND' 'OH'
 'VT' 'WA' 'AZ' 'MO' 'NY' 'PA' 'UT' 'WI']
['AL' 'AK' 'AZ' 'AR' 'CA' 'CO' 'CT' 'FL' 'GA' 'HI' 'ID' 'IL' 'IA' 'KS'
 'ME' 'MD' 'MA' 'MI' 'MN' 'NE' 'NV' 'NH' 'NM' 'NY' 'OH' 'OK' 'OR' 'PA'
 'RI' 'SC' 'SD' 'TN' 'TX' 'VT' 'WI' 'WY' 'UT' 'DE' 'IN' 'MO' 'MT' 'NC'
 'ND' 'WA' 'WV' 'KY' 'LA' 'MS' 'NJ' 'VA']


## Join with...

#### Import

In [401]:
state_unemp = pd.read_csv('./data/state_unemployment.csv')
nat_unemp = pd.read_csv('./data/national_unemployment.csv')

In [402]:
state_unemp.head()

,state_year,unemployment_rate_state_oct
0,AL_1976,7.0
1,AL_1977,6.8
2,AL_1978,6.5
3,AL_1979,7.2
4,AL_1980,9.6


In [403]:
nat_unemp.head()

,month,year,unemployment_rate_national_oct
0,10,1964,5.1
1,10,1965,4.2
2,10,1966,3.7
3,10,1967,4.0
4,10,1968,3.4


In [405]:
sens_w_stateunemp = pd.merge(sens, state_unemp, left_on='loc_date_id', right_on = 'state_year')
sens_w_natunemp = pd.merge(sens_w_stateunemp, nat_unemp, on='year')

govs_w_stateunemp = pd.merge(govs, state_unemp, left_on='loc_date_id', right_on = 'state_year')
govs_w_natunemp = pd.merge(govs_w_stateunemp, nat_unemp, on='year')

In [406]:
sens_w_natunemp.head()

,race_id,office,loc_date_id,state,abbrev,year,GOP_win,winner,rival,pred_GOP,pred_DEM,unopposed,inc_GOP_running,inc_DEM_running,prez_GOP,approval_effects_GOP,approval_effects_DEM,state_year,unemployment_rate_state_oct,month,unemployment_rate_national_oct
0,MN_sen_1996,MN_sen,MN_1996,Minnesota,MN,1996,0,Paul Wellstone Democratic,Rudy Boschwitz,0,0,0,0,1,0.0,0.0,21.0,MN_1996,3.8,10,5.2
1,AL_sen_1996,AL_sen,AL_1996,Alabama,AL,1996,1,Jeff Sessions,Roger Bedford,0,1,0,0,0,0.0,0.0,21.0,AL_1996,5.1,10,5.2
2,AR_sen_1996,AR_sen,AR_1996,Arkansas,AR,1996,1,Tim Hutchinson,Winston Bryant,0,1,0,0,0,0.0,0.0,21.0,AR_1996,5.3,10,5.2
3,DE_sen_1996,DE_sen,DE_1996,Delaware,DE,1996,0,Joe Biden,Raymond J Clatworthy,0,1,0,0,1,0.0,0.0,21.0,DE_1996,5.3,10,5.2
4,GA_sen_1996,GA_sen,GA_1996,Georgia,GA,1996,0,Max Cleland,Guy Millner,0,1,0,0,0,0.0,0.0,21.0,GA_1996,4.8,10,5.2


In [407]:
govs_w_natunemp.head()

,race_id,office,loc_date_id,state,abbrev,year,GOP_win,winner,rival,pred_GOP,pred_DEM,unopposed,inc_GOP_running,inc_DEM_running,prez_GOP,approval_effects_GOP,approval_effects_DEM,state_year,unemployment_rate_state_oct,month,unemployment_rate_national_oct
0,AL_gov_1978,AL_gov,AL_1978,Alabama,AL,1978,0,Fob James,H Guy Hunt,0,1,0,0,0,0.0,0.0,13.0,AL_1978,6.5,10,5.8
1,AK_gov_1978,AK_gov,AK_1978,Alaska,AK,1978,0,Walter Joseph Hickel,Chancy Croft,1,0,0,0,0,0.0,0.0,13.0,AK_1978,10.2,10,5.8
2,AZ_gov_1978,AZ_gov,AZ_1978,Arizona,AZ,1978,1,Evan Mecham,V Gene Lewter,0,1,0,0,0,0.0,0.0,13.0,AZ_1978,5.7,10,5.8
3,AR_gov_1978,AR_gov,AR_1978,Arkansas,AR,1978,0,Bill Clinton,Lynn Lowe,0,1,0,0,0,0.0,0.0,13.0,AR_1978,6.4,10,5.8
4,CA_gov_1978,CA_gov,CA_1978,California,CA,1978,1,Evelle J Younger,Ed Clark,0,1,0,0,0,0.0,0.0,13.0,CA_1978,6.8,10,5.8


In [408]:
sen_states = sens.abbrev.unique()
gov_states = govs.abbrev.unique()
print(len(sen_states))
print(len(gov_states))
print(sen_states)
print(gov_states)

50
50
['MN' 'AL' 'AR' 'DE' 'GA' 'IL' 'IA' 'LA' 'MA' 'MI' 'MT' 'NE' 'NJ' 'RI'
 'WV' 'AK' 'CO' 'ID' 'KS' 'KY' 'ME' 'MS' 'NH' 'NM' 'NC' 'OK' 'OR' 'SC'
 'SD' 'TN' 'TX' 'VA' 'WY' 'HI' 'IN' 'CA' 'CT' 'FL' 'MD' 'NV' 'ND' 'OH'
 'VT' 'WA' 'AZ' 'MO' 'NY' 'PA' 'UT' 'WI']
['AL' 'AK' 'AZ' 'AR' 'CA' 'CO' 'CT' 'FL' 'GA' 'HI' 'ID' 'IL' 'IA' 'KS'
 'ME' 'MD' 'MA' 'MI' 'MN' 'NE' 'NV' 'NH' 'NM' 'NY' 'OH' 'OK' 'OR' 'PA'
 'RI' 'SC' 'SD' 'TN' 'TX' 'VT' 'WI' 'WY' 'UT' 'DE' 'IN' 'MO' 'MT' 'NC'
 'ND' 'WA' 'WV' 'KY' 'LA' 'MS' 'NJ' 'VA']


#### Feature Engineering

In [409]:
def unemployment_engineering(df):
    n_rows = df.shape[0]
    df.drop(['state_year', 'month'], inplace=True, axis=1)
    
    df['nat_UR_effects_GOP'] = 0
    df['nat_UR_effects_DEM'] = 0
    df['state_UR_effects_GOP'] = 0
    df['state_UR_effects_DEM'] = 0
    
    for i in range(n_rows):
        if df.loc[i, 'prez_GOP'] == 1:
            df.loc[i, 'nat_UR_effects_GOP'] = df.loc[i, 'unemployment_rate_national_oct']
            df.loc[i, 'state_UR_effects_GOP'] = df.loc[i, 'unemployment_rate_state_oct']
        else:
            df.loc[i, 'nat_UR_effects_DEM'] = df.loc[i, 'unemployment_rate_national_oct']
            df.loc[i, 'state_UR_effects_DEM'] = df.loc[i, 'unemployment_rate_state_oct']
    
    df.drop(['unemployment_rate_national_oct', 'unemployment_rate_state_oct'], axis=1, inplace=True)
    
    return df

#### Update Values

In [410]:
sens = unemployment_engineering(sens_w_natunemp)
govs = unemployment_engineering(govs_w_natunemp)

In [411]:
sen_states = sens.abbrev.unique()
gov_states = govs.abbrev.unique()
print(len(sen_states))
print(len(gov_states))
print(sen_states)
print(gov_states)

50
50
['MN' 'AL' 'AR' 'DE' 'GA' 'IL' 'IA' 'LA' 'MA' 'MI' 'MT' 'NE' 'NJ' 'RI'
 'WV' 'AK' 'CO' 'ID' 'KS' 'KY' 'ME' 'MS' 'NH' 'NM' 'NC' 'OK' 'OR' 'SC'
 'SD' 'TN' 'TX' 'VA' 'WY' 'HI' 'IN' 'CA' 'CT' 'FL' 'MD' 'NV' 'ND' 'OH'
 'VT' 'WA' 'AZ' 'MO' 'NY' 'PA' 'UT' 'WI']
['AL' 'AK' 'AZ' 'AR' 'CA' 'CO' 'CT' 'FL' 'GA' 'HI' 'ID' 'IL' 'IA' 'KS'
 'ME' 'MD' 'MA' 'MI' 'MN' 'NE' 'NV' 'NH' 'NM' 'NY' 'OH' 'OK' 'OR' 'PA'
 'RI' 'SC' 'SD' 'TN' 'TX' 'VT' 'WI' 'WY' 'UT' 'DE' 'IN' 'MO' 'MT' 'NC'
 'ND' 'WA' 'WV' 'KY' 'LA' 'MS' 'NJ' 'VA']


## Join Census Data

#### Import

In [412]:
census = pd.read_csv('./data/census_final.csv')
census.drop(['year', 'state', 'abbrev', 'combo1'], axis=1, inplace=True)

In [413]:
census.head()

,combo2,total_population,total_male,total_female,age_under18,age_18to29,age_30to59,age_60over,race_white_nh,race_black,race_natamer,race_asian,race_hispanic,percent_male,percent_female,percent_under18,percent_age_18to29,percent_age_30to59,percent_age_60over,percent_race_white,percent_race_black,percent_race_natamer,percent_race_asian,percent_race_hispanic
0,AL_1970,3444165,1661941,1782224,1233520,619880,1115478,475287,2494983,903467,2443,4424,38848,0.482538,0.517462,0.358148,0.179980,0.323875,0.137998,0.724409,0.262318,0.000709,0.001284,0.011279
1,AL_1971,3489137,1682900,1806237,1226393,640938,1133137,488670,2529484,912749,3129,5048,38293,0.482326,0.517674,0.351489,0.183695,0.324761,0.140055,0.724960,0.261597,0.000897,0.001447,0.010975
2,AL_1972,3534110,1703860,1830250,1219266,661997,1150795,502052,2563985,922030,3815,5671,37738,0.482119,0.517881,0.344999,0.187316,0.325625,0.142059,0.725497,0.260895,0.001079,0.001605,0.010678
3,AL_1973,3579082,1724819,1854263,1212138,683055,1168454,515435,2598485,931312,4501,6295,37183,0.481917,0.518083,0.338673,0.190846,0.326468,0.144013,0.726020,0.260210,0.001258,0.001759,0.010389
4,AL_1974,3624054,1745778,1878276,1205011,704113,1186112,528818,2632986,940593,5187,6918,36628,0.481720,0.518280,0.332504,0.194289,0.327289,0.145919,0.726531,0.259542,0.001431,0.001909,0.010107


In [414]:
sens.head()

,race_id,office,loc_date_id,state,abbrev,year,GOP_win,winner,rival,pred_GOP,pred_DEM,unopposed,inc_GOP_running,inc_DEM_running,prez_GOP,approval_effects_GOP,approval_effects_DEM,nat_UR_effects_GOP,nat_UR_effects_DEM,state_UR_effects_GOP,state_UR_effects_DEM
0,MN_sen_1996,MN_sen,MN_1996,Minnesota,MN,1996,0,Paul Wellstone Democratic,Rudy Boschwitz,0,0,0,0,1,0.0,0.0,21.0,0.0,5.2,0.0,3.8
1,AL_sen_1996,AL_sen,AL_1996,Alabama,AL,1996,1,Jeff Sessions,Roger Bedford,0,1,0,0,0,0.0,0.0,21.0,0.0,5.2,0.0,5.1
2,AR_sen_1996,AR_sen,AR_1996,Arkansas,AR,1996,1,Tim Hutchinson,Winston Bryant,0,1,0,0,0,0.0,0.0,21.0,0.0,5.2,0.0,5.3
3,DE_sen_1996,DE_sen,DE_1996,Delaware,DE,1996,0,Joe Biden,Raymond J Clatworthy,0,1,0,0,1,0.0,0.0,21.0,0.0,5.2,0.0,5.3
4,GA_sen_1996,GA_sen,GA_1996,Georgia,GA,1996,0,Max Cleland,Guy Millner,0,1,0,0,0,0.0,0.0,21.0,0.0,5.2,0.0,4.8


In [415]:
sens_w_census = pd.merge(sens, census, left_on='loc_date_id', right_on='combo2')
govs_w_census = pd.merge(govs, census, left_on='loc_date_id', right_on='combo2')

sens_w_census.drop(['combo2', 'total_population'], axis =1, inplace=True)
govs_w_census.drop(['combo2', 'total_population'], axis =1, inplace=True)

In [416]:
govs_w_census.head()

,race_id,office,loc_date_id,state,abbrev,year,GOP_win,winner,rival,pred_GOP,pred_DEM,unopposed,inc_GOP_running,inc_DEM_running,prez_GOP,approval_effects_GOP,approval_effects_DEM,nat_UR_effects_GOP,nat_UR_effects_DEM,state_UR_effects_GOP,state_UR_effects_DEM,total_male,total_female,age_under18,age_18to29,age_30to59,age_60over,race_white_nh,race_black,race_natamer,race_asian,race_hispanic,percent_male,percent_female,percent_under18,percent_age_18to29,percent_age_30to59,percent_age_60over,percent_race_white,percent_race_black,percent_race_natamer,percent_race_asian,percent_race_hispanic
0,AL_gov_1978,AL_gov,AL_1978,Alabama,AL,1978,0,Fob James,H Guy Hunt,0,1,0,0,0,0.0,0.0,13.0,0.0,5.8,0.0,6.5,1829615,1974328,1176502,788346,1256746,582349,2770988,977720,7932,9413,34409,0.480979,0.519021,0.309285,0.207244,0.330380,0.153091,0.728452,0.257028,0.002085,0.002475,0.009046
1,AK_gov_1978,AK_gov,AK_1978,Alaska,AK,1978,0,Walter Joseph Hickel,Chancy Croft,1,0,0,0,0,0.0,0.0,13.0,0.0,5.8,0.0,10.2,203084,178473,128568,101180,133656,18154,288403,12781,54741,14337,8525,0.532251,0.467749,0.336956,0.265177,0.350291,0.047579,0.755858,0.033497,0.143467,0.037575,0.022343
2,AZ_gov_1978,AZ_gov,AZ_1978,Arizona,AZ,1978,1,Evan Mecham,V Gene Lewter,0,1,0,0,0,0.0,0.0,13.0,0.0,5.8,0.0,5.7,1244555,1284197,761985,549484,825128,392155,1723705,69996,142674,23009,405515,0.492162,0.507838,0.301328,0.217295,0.326299,0.155078,0.681643,0.027680,0.056421,0.009099,0.160362
3,AR_gov_1978,AR_gov,AR_1978,Arkansas,AR,1978,0,Bill Clinton,Lynn Lowe,0,1,0,0,0,0.0,0.0,13.0,0.0,5.8,0.0,6.4,1070212,1143595,668101,427351,713077,405278,1805936,368909,10608,6370,19195,0.483426,0.516574,0.301788,0.193039,0.322104,0.183068,0.815760,0.166640,0.004792,0.002877,0.008671
4,CA_gov_1978,CA_gov,CA_1978,California,CA,1978,1,Evelle J Younger,Ed Clark,0,1,0,0,0,0.0,0.0,13.0,0.0,5.8,0.0,6.8,11296525,11628423,6438361,5169058,8077838,3239692,14019966,1734957,203565,1190567,4109323,0.492761,0.507239,0.280845,0.225477,0.352360,0.141317,0.611559,0.075680,0.008880,0.051933,0.179251


In [417]:
govs = govs_w_census
sens = sens_w_census

In [418]:
govs.to_csv('./data/govs_final.csv')
sens.to_csv('./data/sens_final.csv')